In [1]:
import numpy as np
from itertools import product
import itertools
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mindreadingautobots.entropy_and_bayesian import boolean


### Find a trick (Joker) function 

Recall that we start with uniformly random bitstrings $X$ (each length $n$ bitstrings), then we generate $Y=f(X)$ and create a pair $(X, Y)$. This pair $X,Y$ has a joint distribution. By applying bitflips to turn $X \rightarrow Z$, we end up with a new joinst distr $p_{ZY}$ where $Z$ is a bitflipped version of $X$ and $Y=f(X)$.

By trick function, we mean that for data with  we are looking for $g^*$ (optimal prediction for _noisy_ data) is more accuracte and less sensitive than $f$ (function used to generate noiseless data).

We want an example of data and noise such that MLD (for the noisy data) evaluated on noiseless data is _worse_ than MLD (for noiseless data) evaluated on noiseless data.

Suppose $f^*(z^{n-1}) = \argmax_{x'} p_{X|Z^{n-1}}(x|z^{n-1})$ is our MLD for noisy data and $fN*(x^{n-1}) = \argmax_{x'} p_{X|X^{n-1}}(x|x^{n-1})$ is our MLD for noiseless data. We build $f^*$ analytically, then compare it to $g^*$. 

Our example will be $k=3$ majority function (in which case $g^*$ is just majority).

#### Searching for weight-based counterexamples

Instead of defining boolean functions as $f:\{0,1\}^n \rightarrow \{0,1\}$, we define $f$ such that 
$$
f(x) = g(\text{wt}(x))
$$
what this does is, there are only $2^{n+1}$ possible functions $g$, instead of $2^{2^n}$ possible functions $f$. For example, if $n=3$, then a possible $g$ is 
\begin{equation}
     \text{wt}(x) \rightarrow \begin{cases}
        0 \rightarrow 0 \\
        1 \rightarrow 1 \\
        2 \rightarrow 0 \\
        3 \rightarrow 1
    \end{cases}
\end{equation}

### Searching for joker functions

**constraints**:
- we want both senstivities to be $\gg 0$
- we want function accuracies to be $\gg 1/2$ (this is equivalent to $p$ not big)
- we want functions that are more balanced (`imbal` closer to 0)

In [4]:
n = 5 # total number of bits
p = 0.3 # probability of each bit of X flipping, this DOESN'T flip Y ever.


X_arr = np.array(list(itertools.product([0, 1], repeat=n)))
# p_x = 1 / (2 ** n) # uniform distribution over x # chaos distribution and the thing with chaos distribution is its fair
# WEIGHT-BASED FUNCTIONS
signatures = itertools.product([0, 1], repeat=n+1)
for signature in signatures:
# signature = [1,0,0,1,1,0] # n+1 elements
    print("counterexample=", signature)
    hash = dict(zip(range(n+1), signature))
    func = lambda b: hash[sum(b)]

    f_accs = []
    fn_accs = []
    # noisy_lookup[row,col] is the JOINT probability Pr(f(z)=row| x=col)
    noisy_lookup = np.zeros((2, 2**n))
    true_lookup = np.zeros((2, 2**n))
    # simulate a noisy dataset essentially
    for i, x in enumerate(X_arr):
        func_value = func(x) # compute y=f(x)
        # true lookup is an array with 2 rows; there is a p_x at [row, column] if 
        # f[column] = row]. so, true_lookup[i, j] = pr(f(x) = i| x=j)
        true_lookup[func(x), i] = 1
        # iterate over all of the z values that contribute to 
        for e in product([0, 1], repeat=n):
            z = np.array(x) ^ np.array(e)
            p_x_given_z = p ** sum(e) * (1-p)**(n - sum(e))
            # increment noisy_lookup at the binary index of z
            # noisy_lookup[i, j] = pr(f(z) = i,  x=j) 
            noisy_lookup[func_value, int(''.join(map(str, z)), 2)] += p_x_given_z 

    # the function is balanced if the sums of the two rows of true_lookup are equal
    # GOAL: f should not be too imbalanced <=> `imbal` should be close to 0
    imbal = abs(true_lookup[0,:].sum() - true_lookup[1,:].sum())  / 2 ** n

    # if not balanced:
    #     continue
    # round up to get argmax 
    noisy_mle = np.round(noisy_lookup)  
    out = np.multiply(noisy_mle, true_lookup) / 2 ** n # "inner product" of the functions
    diff = out.sum()


    fnstar_dct = {}
    for i, x in enumerate(X_arr):
        fnstar_dct[tuple(x)] = np.argmax(noisy_lookup[:, i])
    def fnstar(x):
        return fnstar_dct[tuple(x)]

    sensitivity_f = boolean.average_sensitivity(func, X_arr)
    sensitivity_fnstar = boolean.average_sensitivity(fnstar, X_arr)
    sensitivity_diff = sensitivity_f - sensitivity_fnstar
    # accuracies on dataset
    p_zy = boolean.generate_noisy_distr(n, p, func)
    noisy_f_acc = boolean.compute_acc_noisytest(p_zy, func, n) # accuracy of f on noisy data
    noiseless_fnstar_acc = boolean.compute_acc_test(fnstar, func, n) # accuracy of fN* on noiseless data
    noisy_fnstar_acc = boolean.compute_acc_noisytest(p_zy, fnstar, n) # accuracy of fN* MLE on noisy data

    acc_diff = noisy_fnstar_acc - noisy_f_acc
    f_accs.append(noisy_f_acc)
    fn_accs.append(noisy_fnstar_acc)

    # print("\t noisy fN* acc=", fn_accs)
    # print("\t noisy fN* sensitivity=", sensitivity_fnstar)

    # print("\t noisy f acc=", f_accs)
    # print("\t noisy f sensitivity=", sensitivity_f)

    # print((fn_accs[0], sensitivity_fnstar), (f_accs[0], sensitivity_f))
    # print()



counterexample= [0, 0, 0, 1, 1, 1]
	 noisy g* acc= [np.float64(0.6463449999999998)]
	 noisy g* sensitivity= 1.875
	 noisy f acc= [np.float64(0.6463449999999998)]
	 noisy f sensitivity= 1.875
(np.float64(0.6463449999999998), 1.875) (np.float64(0.6463449999999998), 1.875)



### Checking even vs odd majority functions

In [11]:
n = 5 # total number of bits
p = 0.3 # probability of each bit of X flipping, this DOESN'T flip Y ever.


# p_x = 1 / (2 ** n) # uniform distribution over x # chaos distribution and the thing with chaos distribution is its fair
# WEIGHT-BASED FUNCTIONS
signatures = [[0, 0, 0, 1, 1, 1], [0, 0, 1, 1, 1]]
# signatures = itertools.product([0, 1], repeat=n+1)
for signature in signatures:
    n = len(signature) - 1
    X_arr = np.array(list(itertools.product([0, 1], repeat=n)))
    print("counterexample=", signature)
    hash = dict(zip(range(n+1), signature))
    func = lambda b: hash[sum(b)]

    f_accs = []
    fn_accs = []
    # noisy_lookup[row,col] is the JOINT probability Pr(f(z)=row| x=col)
    noisy_lookup = np.zeros((2, 2**n))
    true_lookup = np.zeros((2, 2**n))
    # simulate a noisy dataset essentially
    for i, x in enumerate(X_arr):
        func_value = func(x) # compute y=f(x)
        # true lookup is an array with 2 rows; there is a p_x at [row, column] if 
        # f[column] = row]. so, true_lookup[i, j] = pr(f(x) = i| x=j)
        true_lookup[func(x), i] = 1
        # iterate over all of the z values that contribute to 
        for e in product([0, 1], repeat=n):
            z = np.array(x) ^ np.array(e)
            p_x_given_z = p ** sum(e) * (1-p)**(n - sum(e))
            # increment noisy_lookup at the binary index of z
            # noisy_lookup[i, j] = pr(f(z) = i,  x=j) 
            noisy_lookup[func_value, int(''.join(map(str, z)), 2)] += p_x_given_z 

    # the function is balanced if the sums of the two rows of true_lookup are equal
    # GOAL: f should not be too imbalanced <=> `imbal` should be close to 0
    imbal = abs(true_lookup[0,:].sum() - true_lookup[1,:].sum())  / 2 ** n

    # round up to get argmax 
    noisy_mle = np.round(noisy_lookup)  
    out = np.multiply(noisy_mle, true_lookup) / 2 ** n # "inner product" of the functions
    diff = out.sum()

    fnstar_dct = {}
    for i, x in enumerate(X_arr):
        fnstar_dct[tuple(x)] = np.argmax(noisy_lookup[:, i])
    def fnstar(x):
        return fnstar_dct[tuple(x)]

    sensitivity_f = boolean.average_sensitivity(func, X_arr)
    sensitivity_fnstar = boolean.average_sensitivity(fnstar, X_arr)
    sensitivity_diff = sensitivity_f - sensitivity_fnstar
    # accuracies on dataset
    p_zy = boolean.generate_noisy_distr(n, p, func)
    noisy_f_acc = boolean.compute_acc_noisytest(p_zy, func, n) # accuracy of f on noisy data
    noiseless_fnstar_acc = boolean.compute_acc_test(fnstar, func, n) # accuracy of fN* on noiseless data
    noisy_fnstar_acc = boolean.compute_acc_noisytest(p_zy, fnstar, n) # accuracy of fN* MLE on noisy data

    acc_diff = noisy_fnstar_acc - noisy_f_acc
    f_accs.append(noisy_f_acc)
    fn_accs.append(noisy_fnstar_acc)
    print("noisy val acc of noisy MLE")
    print("\t noisy fN* acc=", fn_accs)
    print("\t fN* sensitivity=", sensitivity_fnstar)
    print("noisy val acc of f")
    print("\t noisy f acc=", f_accs)
    print("\t f sensitivity=", sensitivity_f)
    print("noiseless val acc of gfN*")
    print("\t noiseless fN* acc=", noiseless_fnstar_acc)


    print((fn_accs[0], sensitivity_fnstar), (f_accs[0], sensitivity_f))
    print()



counterexample= [0, 0, 0, 1, 1, 1]
noisy val acc of noisy MLE
	 noisy fN* acc= [np.float64(0.6463449999999998)]
	 fN* sensitivity= 1.875
noisy val acc of f
	 noisy f acc= [np.float64(0.6463449999999998)]
	 f sensitivity= 1.875
noiseless val acc of gfN*
	 noiseless fN* acc= 1.0
(np.float64(0.6463449999999998), 1.875) (np.float64(0.6463449999999998), 1.875)

counterexample= [0, 0, 1, 1, 1]
noisy val acc of noisy MLE
	 noisy fN* acc= [np.float64(0.7064624999999999)]
	 fN* sensitivity= 0.5
noisy val acc of f
	 noisy f acc= [np.float64(0.6941124999999999)]
	 f sensitivity= 1.5
noiseless val acc of gfN*
	 noiseless fN* acc= 0.75
(np.float64(0.7064624999999999), 0.5) (np.float64(0.6941124999999999), 1.5)



### Different search: not weight based.

In [ ]:
# this code looks at all possible boolean functions that are perfectly balanced

def boolean_function_from_signature(f):
    """Given a length 2^n binary array, return the function with that boolean signature."""
    X_arr = list(itertools.product([0, 1], repeat=n))
    X_arr = [tuple(x) for x in X_arr]
    lookup = dict(zip(X_arr, f))
    def func(x):
        return lookup[tuple(x)]
    return lookup, func

n = 4
# WARNING: n=4 might take 5 minutes
assert n <= 4
k = n
# pvals = [0.01, 0.25, 0.49]
pvals = [0.2]

# a "signature" of a boolean function is a length 2**n bitstring S where f(x) = S[bin(x)]
# we will check signatures for perfectly balanced functions
sig_arr = np.array(list(itertools.product([0, 1], repeat=2**n)))
sig_arr = sig_arr[sig_arr.sum(axis=1) == 2**(n-1)] # only balanced functions

X_arr = np.array(list(itertools.product([0, 1], repeat=n)))
p_x = 1 / (2 ** k) # uniform distribution over x
for i, signature in enumerate(sig_arr):
    # iterate over signatures
    print(signature)
    dct, func = boolean_function_from_signature(signature)
    for p in pvals:        
        # noisy_lookup[row,col] is the JOINT probability Pr(f(z)=row| x=col)
        noisy_lookup = np.zeros((2, 2**n))
        true_lookup = np.zeros((2, 2**n))
        # simulate a noisy dataset essentially
        for i, x in enumerate(product([0,1], repeat=k)):
            func_value = func(x)
            # true lookup is an array with 2 rows; there is a p_x at [row, column] if 
            # f[column] = row]. so, true_lookup[i, j] = pr(f(x) = i| x=j)
            true_lookup[func(x), i] = 1
            # iterate over all of the z values that contribute to 
            for e in product([0, 1], repeat=k):
                z = np.array(x) ^ np.array(e)
                p_x_given_z = p ** sum(e) * (1-p)**(k - sum(e))
                # increment noisy_lookup at the binary index of z
                # noisy_lookup[i, j] = pr(f(z) = i,  x=j) 
                noisy_lookup[func_value, int(''.join(map(str, z)), 2)] += p_x_given_z 
        
        # the function is balanced if the sums of the two rows of true_lookup are equal
        # imbal = abs(true_lookup[0,:].sum() - true_lookup[1,:].sum())  / 2 ** n
        # round up to get argmax 
        noisy_mle = np.round(noisy_lookup)  
        out = np.multiply(noisy_mle, true_lookup) / 2 ** n # "inner product" of the functions
        diff = out.sum()
        fnstar_dct = {}
        for i, x in enumerate(X_arr):
            fnstar_dct[tuple(x)] = np.argmax(noisy_lookup[:, i])
        def fnstar(x):
            return fnstar_dct[tuple(x)]
        
        sensitivity_f = boolean.average_sensitivity(func, X_arr)
        sensitivity_fnstar = boolean.average_sensitivity(fnstar, X_arr)
        sensitivity_diff = sensitivity_f - sensitivity_fnstar
        # accuracies on dataset
        p_zy = boolean.generate_noisy_distr(k, p, func)
        noisy_f_acc = boolean.compute_acc_noisytest(p_zy, func, n) # accuracy of f on noisy data
        # noiseless_fnstar_acc = compute_acc_test(fnstar, func, n) # accuracy of fN* on noiseless data
        noisy_fnstar_acc = boolean.compute_acc_noisytest(p_zy, fnstar, n) # accuracy of fN* MLE on noisy data


#### Random search over boolean functions

In [ ]:
# n = 5
n = 5
k = n
# p = 0.2
pvals = [0, 0.1, 0.15, 0.2, 0.25, 0.3]

X_arr = np.array(list(itertools.product([0, 1], repeat=n)))


p_x = 1 / (2 ** k) # uniform distribution over x

H = np.array(list(itertools.product([0, 1], repeat=n+1)))
mine = [
    [1,0,0,1,1,0],
    [0,1,1,0,1,0],
    [0,1,1,0,0,0]
]
# H = [[0, 1, 0, 0, 0]]

for p in pvals:
    print()
    print(f"!!!p={p}")
    print("boolean signature | imbal? | nacc(fN*) | taccfN* |naccfN*-naccf| S(f) | S(fN*) | S(f) - S(fN*)")
    print("--------------------------------------------------------------------------")
    for i, signature in enumerate(H):
        sss = "  "
        if list(signature) in mine:
            sss = ">>"
        hash = dict(zip(range(n+1), signature))
        func = lambda b: hash[sum(b)]
        
        # noisy_lookup[row,col] is the JOINT probability Pr(f(z)=row| x=col)
        noisy_lookup = np.zeros((2, 2**n))
        true_lookup = np.zeros((2, 2**n))
        # simulate a noisy dataset essentially
        for i, x in enumerate(product([0,1], repeat=k)):
            func_value = func(x)
            # true lookup is an array with 2 rows; there is a p_x at [row, column] if 
            # f[column] = row]. so, true_lookup[i, j] = pr(f(x) = i| x=j)
            true_lookup[func(x), i] = 1
            # iterate over all of the z values that contribute to 
            for e in product([0, 1], repeat=k):
                z = np.array(x) ^ np.array(e)
                p_x_given_z = p ** sum(e) * (1-p)**(k - sum(e))
                # increment noisy_lookup at the binary index of z
                # noisy_lookup[i, j] = pr(f(z) = i,  x=j) 
                noisy_lookup[func_value, int(''.join(map(str, z)), 2)] += p_x_given_z 
        
        # the function is balanced if the sums of the two rows of true_lookup are equal
        imbal = abs(true_lookup[0,:].sum() - true_lookup[1,:].sum())  / 2 ** n
        
        # if not balanced:
        #     continue
        # round up to get argmax 
        noisy_mle = np.round(noisy_lookup)  
        out = np.multiply(noisy_mle, true_lookup) / 2 ** n # "inner product" of the functions
        noiseless_fnstar_acc = out.sum()


        fnstar_dct = {}
        for i, x in enumerate(X_arr):
            fnstar_dct[tuple(x)] = np.argmax(noisy_lookup[:, i])
        def fnstar(x):
            return fnstar_dct[tuple(x)]
        
        sensitivity_f = average_sensitivity(func, X_arr)
        sensitivity_fnstar = average_sensitivity(fnstar, X_arr)
        sensitivity_diff = sensitivity_f - sensitivity_fnstar
        # accuracies on dataset
        #  = compute_acc_test(fnstar, func, n) # accuracy of fN* on noiseless data

        p_zy = generate_noisy_distr(k, p, func)
        noisy_f_acc = compute_acc_noisytest(p_zy, func, n) # accuracy of f on noisy data
        noisy_fnstar_acc = compute_acc_noisytest(p_zy, fnstar, n) # accuracy of fN* MLE on noisy data
        nacc_diff = noisy_fnstar_acc - noisy_f_acc



        print(f"{sss}{signature}   | {imbal:0.4f} |   {noisy_fnstar_acc:1.4f}  | {noiseless_fnstar_acc:1.4f} | {nacc_diff:1.4f}    |{sensitivity_f:1.4f}|{sensitivity_fnstar:1.4f}  |  {sensitivity_diff:1.4f} ")
        if sensitivity_fnstar == 0:
            topr = sum(noisy_mle[0,:])
            botr = sum(noisy_mle[1,:])
            assert (np.allclose(topr, 0) or np.allclose(topr, 1 << n))
            assert (np.allclose(botr, 0) or np.allclose(botr, 1 << n))

    
    



!!!p=0
boolean signature | imbal? | nacc(fN*) | taccfN* |naccfN*-naccf| S(f) | S(fN*) | S(f) - S(fN*)
--------------------------------------------------------------------------
  [0 0 0 0 0 0]   | 1.0000 |   1.0000  | 1.0000 | 0.0000    |0.0000|0.0000  |  0.0000 
  [0 0 0 0 0 1]   | 0.9375 |   1.0000  | 1.0000 | 0.0000    |0.3125|0.3125  |  0.0000 
  [0 0 0 0 1 0]   | 0.6875 |   1.0000  | 1.0000 | 0.0000    |1.5625|1.5625  |  0.0000 
  [0 0 0 0 1 1]   | 0.6250 |   1.0000  | 1.0000 | 0.0000    |1.2500|1.2500  |  0.0000 
  [0 0 0 1 0 0]   | 0.3750 |   1.0000  | 1.0000 | 0.0000    |3.1250|3.1250  |  0.0000 
  [0 0 0 1 0 1]   | 0.3125 |   1.0000  | 1.0000 | 0.0000    |3.4375|3.4375  |  0.0000 
  [0 0 0 1 1 0]   | 0.0625 |   1.0000  | 1.0000 | 0.0000    |2.1875|2.1875  |  0.0000 
  [0 0 0 1 1 1]   | 0.0000 |   1.0000  | 1.0000 | 0.0000    |1.8750|1.8750  |  0.0000 
  [0 0 1 0 0 0]   | 0.3750 |   1.0000  | 1.0000 | 0.0000    |3.1250|3.1250  |  0.0000 
  [0 0 1 0 0 1]   | 0.3125 |   1.0000  